In [ ]:
#---- Install stuff -------
!pip install --upgrade pip

!pip install opencv-python
!pip install Pillow

!pip install playsound
!pip install gTTS
#---- Install stuff -------


In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from glob import glob
from pathlib import Path

import cv2
from PIL import Image
import matplotlib.pyplot as plt

from gtts import gTTS
from playsound import playsound
import random

In [ ]:
from IPython.display import clear_output

In [ ]:
import pycode.FileIOUtil
import pycode.TTSUtil

from pycode.ImageClassificationCfg import ImageClassificationCfg
from pycode.LoadAndTestCNNUtil import LoadAndTestCNNUtil

In [ ]:
#print system info
import sys
print('Python: ',sys.version)

In [ ]:
is_google_colab = True

In [ ]:
## If you are using the data by mounting the google drive, use the following :
if is_google_colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    ##Ref:https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166

In [ ]:
#read cfg file
if is_google_colab:
  cfg_file = '/content/gdrive/MyDrive/akshath/cfg.yml'

cfg_file = ImageClassificationCfg(cfg_file)
cfg_file.load()

cfg_file.log_info()

In [ ]:
data_dir = './data/'

if is_google_colab:
  #data dir
  data_dir = '/content/gdrive/MyDrive/Colab Notebooks/Hackathon/project/pigeons_detection/data/'
  !ls '$data_dir'

In [ ]:
all_images = glob(data_dir+'*/*.png')

In [ ]:
len(all_images)

In [ ]:
all_images[0]

In [ ]:
#lets get a good sample
img_file = random.choice(all_images)
print(img_file)

good_image = cv2.imread(img_file, 1)
print(good_image.shape)
plt.imshow(good_image)
plt.show()

In [ ]:
img_h = good_image.shape[0]
img_w = good_image.shape[1]
print('good dim h:',img_h)
print('good dim w:',img_w)

In [ ]:
#select image to work with

In [ ]:
while True:
  img_file = random.choice(all_images)
  frame = cv2.imread(img_file, 1)
    
  if frame.shape[0]!=360:
    break
print(img_file)
print(frame.shape)
plt.imshow(frame)
plt.show()

In [ ]:
def fix_image(frame, filename):
  img_h = frame.shape[0]
  img_w = frame.shape[1]
  #print('current dim h:',img_h)
  #print('current dim w:',img_w)

  h = img_h//cfg_file.cfg['reduce_image_wh_by']
  w = img_w//cfg_file.cfg['reduce_image_wh_by']
  #print('new dim h:',h)
  #print('new dim w:',w)

  frame = cv2.resize(frame, (w,h), interpolation = cv2.INTER_AREA)

  new_wl = int(w*cfg_file.cfg['crop_image_from_left'])
  new_wr = int(w*cfg_file.cfg['crop_image_from_right'])

  if cfg_file.cfg['crop_image_from_left']>0:
      new_wl = int(w*cfg_file.cfg['crop_image_from_left'])
  else:
      new_wl = 0
      
  if cfg_file.cfg['crop_image_from_right']>0:
      #crop 70% on width from right
      new_wr = int(w*cfg_file.cfg['crop_image_from_right'])
  else:
      new_wr = w                    
      
  frame = frame[0:h,new_wl:new_wr]

  #print(frame.shape)
  #plt.imshow(frame)
  #plt.show()
  
  cv2.imwrite(img_file, frame)

In [ ]:
fix_image(frame, img_file)

In [ ]:
#test again
fixed_image = cv2.imread(img_file, 1)
print(fixed_image.shape)
plt.imshow(fixed_image)
plt.show()

In [ ]:
#lets do for all
count = 0
print(f'Total images: {len(all_images)}')
for img_file in all_images:
  frame = cv2.imread(img_file, 1)
    
  if frame.shape[0]!=360:
    fix_image(frame, img_file)
    count += 1
print(f'Fixed {count} images')

In [ ]:
#!zip -r '/content/gdrive/MyDrive/Colab Notebooks/Hackathon/project/pigeons_detection.zip' '$data_dir'